In [1]:
# 4_evaluation.ipynb

# ---
# 目的: 载入训练好的模型 => 评估在测试集(或相同样本) => 计算准确率等

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

MODEL_PATH = "../saved_models/mlp_chorus_model.pt"
FEATURE_FOLDER = "../data/processed/"

# 同一个 MLP 结构
class MLP(nn.Module):
    def __init__(self, input_dim=13, hidden_dim=16, output_dim=2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = MLP(input_dim=13, hidden_dim=16, output_dim=2)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# 加载 feature_index.csv
df = pd.read_csv(os.path.join(FEATURE_FOLDER, "feature_index.csv"))
X_list = []
y_list = []
for idx, row in df.iterrows():
    feats = np.load(row["feature_path"])
    X_list.append(feats)
    label = 0 if idx==0 else 1  # demo
    y_list.append(label)

X = np.array(X_list, dtype=np.float32)
y = np.array(y_list, dtype=np.int64)

X_t = torch.from_numpy(X)
with torch.no_grad():
    output = model(X_t)  # shape (N,2)
pred = output.argmax(dim=1).numpy()

acc = (pred == y).mean()
print("Prediction:", pred, " Label:", y)
print(f"Accuracy = {acc*100:.2f}%")

# (如果你只有2首歌, 这里评估其实没什么意义, 只是演示流程)